# Using Time Series to more accurately predict wave heights

In [ ]:
import utilities.misc as misc
import utilities.metrics as mtr
import utilities.ml_functions as ml

In [ ]:
# convert datetime to unix

In [ ]:
# plot data in time 

In [ ]:
# predict using daily seasonality